In this notebook I am going to cover the processing of cleaning and formatting a specific dataset for EMADE.

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import shutil
import gzip

import os
import re
import math

We will start by importing all the libraries we will need. The most important libraries are numpy and pandas because we will use their data types to store and manipulate our dataset.

Note: Make sure you have nltk's corpus downloaded. You can check by running nltk.download() in a Python shell.

Next, we will import our dataset into a pandas dataframe object.

In [2]:
data = pd.read_csv("datasets/winemag-data_first150k.csv")

# Print first 5 examples
print(data.head())
print(data.shape)

# remove all rows except for the first 25,000
data = data.drop(data.index[25000:])
print(data.shape)

  country                                        description  \
0      US  This tremendous 100% varietal wine hails from ...   
1   Spain  Ripe aromas of fig, blackberry and cassis are ...   
2      US  Mac Watson honors the memory of a wine once ma...   
3      US  This spent 20 months in 30% new French oak, an...   
4  France  This is the top wine from La Bégude, named aft...   

                            designation  points  price        province  \
0                     Martha's Vineyard      96  235.0      California   
1  Carodorum Selección Especial Reserva      96  110.0  Northern Spain   
2         Special Selected Late Harvest      96   90.0      California   
3                               Reserve      96   65.0          Oregon   
4                            La Brûlade      95   66.0        Provence   

            region_1           region_2             variety  \
0        Napa Valley               Napa  Cabernet Sauvignon   
1               Toro                NaN     

The original link to the dataset can be found here: https://www.kaggle.com/zynicide/wine-reviews

However, I removed the index column from the csv in Excel.

The dataset consists of 10 features. Most of these are string values, which machine learning algorithms and data transformations will not accept. The dataset also has columns with missing values. We will either have to remove those columns or replace the missing values.

We also cut the dataset size down to 25,000 examples to reduce the size of our final dataset and RAM usage.

Note: if your computer does not have at least 8 GB of RAM, you might need to reduce the size of the data even more

Next, we will separate a column for labels and modify its values.

In [3]:
labels = data[["price"]]
print(labels.shape)

# Fill in all missing values with mean of column
labels = labels.fillna(labels.mean())

# Set all values greater or equal to 50 to 1
labels.loc[labels.price >= 50, 'price'] = 1
labels.loc[labels.price != 1, 'price'] = 0

(25000, 1)


In this case, we will choose to classify whether a wine has a price greater than or equal to 50. It is important to learn how to recognize potential labels and classification problems when you find a dataset. Most datasets will not have pre-defined labels and even those with labels you can tweak into a different problem.

We also fill in the missing price values with the mean of all price values. There are many different methods of dealing with missing values. You could remove all missing value rows or replace missing values with the median instead.

Next, we will remove irrelevant and/or unuseful features.

In [4]:
data = data.drop("price", axis=1)
data = data.drop("region_2", axis=1)
data = data.drop("winery", axis=1)
data = data.drop("designation", axis=1)

I removed these features for a specific reason. Price needs to be removed because we will add it back as labels later. Region 2 is similar enough to region_1 to not add useful information. Winery and designation have too many unique values.

When we one-hot encode some of our features in the next section, too many unique values will matter. If most of the strings in a feature are unique, we get very little useful or relevant information and only make the dimensions of our data unnecessarily large. This also applies to region 2. We do not want to double our dimensions with overlapping information.

Next, we will one-hot encoding on our remaining string features to extract relevant information from them. 

In [5]:
data = pd.get_dummies(data, columns=["country", "province", "variety", "region_1"])

An explanation of one-hot encoding can be found here: https://www.quora.com/What-is-one-hot-encoding-and-when-is-it-used-in-data-science

Next, we will separate the descriptions column and clean up the text data. One-hot encoding does not work well on columns with unique sentences or paragraphs because all of the items in the column will end up being unique.

In [6]:
text = data[["description"]].values
data = data.drop("description", axis=1)

text_list = []
for i in text:
    text_list.append(i[0])
    
# Alternative
# text = text.tolist()
# text = [i[0] for i in text]
    
# For debugging
print(text_list[0])
count = 0

stemmer = PorterStemmer()

for text in text_list:
    count += 1
    text = text.lower()
    text = re.sub('[!@#$.,?]', '', text)
    words = text.split(" ")
    words = [word for word in words if word not in stopwords.words('english')]
    words = [stemmer.stem(word) for word in words]
    text = " ".join(words)
    print("line " + str(count) + " processed")

This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022–2030.
line 1 processed
line 2 processed
line 3 processed
line 4 processed
line 5 processed
line 6 processed
line 7 processed
line 8 processed
line 9 processed
line 10 processed
line 11 processed
line 12 processed
line 13 processed
line 14 processed
line 15 processed
line 16 processed
line 17 processed
line 18 processed
line 19 processed
line 20 processed
line 21 processed
line 22 processed
line 23 processed
line 24 processed
line 25 processed
line 26 processed
line 27 processed
line 28 processed
line 29 processed
line 30 processed
line 31 processed
line 32 processed
line 33 processed
line 34 processed
line 35 processed
line 36 processed
line 

line 439 processed
line 440 processed
line 441 processed
line 442 processed
line 443 processed
line 444 processed
line 445 processed
line 446 processed
line 447 processed
line 448 processed
line 449 processed
line 450 processed
line 451 processed
line 452 processed
line 453 processed
line 454 processed
line 455 processed
line 456 processed
line 457 processed
line 458 processed
line 459 processed
line 460 processed
line 461 processed
line 462 processed
line 463 processed
line 464 processed
line 465 processed
line 466 processed
line 467 processed
line 468 processed
line 469 processed
line 470 processed
line 471 processed
line 472 processed
line 473 processed
line 474 processed
line 475 processed
line 476 processed
line 477 processed
line 478 processed
line 479 processed
line 480 processed
line 481 processed
line 482 processed
line 483 processed
line 484 processed
line 485 processed
line 486 processed
line 487 processed
line 488 processed
line 489 processed
line 490 processed
line 491 pro

line 875 processed
line 876 processed
line 877 processed
line 878 processed
line 879 processed
line 880 processed
line 881 processed
line 882 processed
line 883 processed
line 884 processed
line 885 processed
line 886 processed
line 887 processed
line 888 processed
line 889 processed
line 890 processed
line 891 processed
line 892 processed
line 893 processed
line 894 processed
line 895 processed
line 896 processed
line 897 processed
line 898 processed
line 899 processed
line 900 processed
line 901 processed
line 902 processed
line 903 processed
line 904 processed
line 905 processed
line 906 processed
line 907 processed
line 908 processed
line 909 processed
line 910 processed
line 911 processed
line 912 processed
line 913 processed
line 914 processed
line 915 processed
line 916 processed
line 917 processed
line 918 processed
line 919 processed
line 920 processed
line 921 processed
line 922 processed
line 923 processed
line 924 processed
line 925 processed
line 926 processed
line 927 pro

line 1297 processed
line 1298 processed
line 1299 processed
line 1300 processed
line 1301 processed
line 1302 processed
line 1303 processed
line 1304 processed
line 1305 processed
line 1306 processed
line 1307 processed
line 1308 processed
line 1309 processed
line 1310 processed
line 1311 processed
line 1312 processed
line 1313 processed
line 1314 processed
line 1315 processed
line 1316 processed
line 1317 processed
line 1318 processed
line 1319 processed
line 1320 processed
line 1321 processed
line 1322 processed
line 1323 processed
line 1324 processed
line 1325 processed
line 1326 processed
line 1327 processed
line 1328 processed
line 1329 processed
line 1330 processed
line 1331 processed
line 1332 processed
line 1333 processed
line 1334 processed
line 1335 processed
line 1336 processed
line 1337 processed
line 1338 processed
line 1339 processed
line 1340 processed
line 1341 processed
line 1342 processed
line 1343 processed
line 1344 processed
line 1345 processed
line 1346 processed


line 1721 processed
line 1722 processed
line 1723 processed
line 1724 processed
line 1725 processed
line 1726 processed
line 1727 processed
line 1728 processed
line 1729 processed
line 1730 processed
line 1731 processed
line 1732 processed
line 1733 processed
line 1734 processed
line 1735 processed
line 1736 processed
line 1737 processed
line 1738 processed
line 1739 processed
line 1740 processed
line 1741 processed
line 1742 processed
line 1743 processed
line 1744 processed
line 1745 processed
line 1746 processed
line 1747 processed
line 1748 processed
line 1749 processed
line 1750 processed
line 1751 processed
line 1752 processed
line 1753 processed
line 1754 processed
line 1755 processed
line 1756 processed
line 1757 processed
line 1758 processed
line 1759 processed
line 1760 processed
line 1761 processed
line 1762 processed
line 1763 processed
line 1764 processed
line 1765 processed
line 1766 processed
line 1767 processed
line 1768 processed
line 1769 processed
line 1770 processed


...

line 23022 processed
line 23023 processed
line 23024 processed
line 23025 processed
line 23026 processed
line 23027 processed
line 23028 processed
line 23029 processed
line 23030 processed
line 23031 processed
line 23032 processed
line 23033 processed
line 23034 processed
line 23035 processed
line 23036 processed
line 23037 processed
line 23038 processed
line 23039 processed
line 23040 processed
line 23041 processed
line 23042 processed
line 23043 processed
line 23044 processed
line 23045 processed
line 23046 processed
line 23047 processed
line 23048 processed
line 23049 processed
line 23050 processed
line 23051 processed
line 23052 processed
line 23053 processed
line 23054 processed
line 23055 processed
line 23056 processed
line 23057 processed
line 23058 processed
line 23059 processed
line 23060 processed
line 23061 processed
line 23062 processed
line 23063 processed
line 23064 processed
line 23065 processed
line 23066 processed
line 23067 processed
line 23068 processed
line 23069 pr

line 23416 processed
line 23417 processed
line 23418 processed
line 23419 processed
line 23420 processed
line 23421 processed
line 23422 processed
line 23423 processed
line 23424 processed
line 23425 processed
line 23426 processed
line 23427 processed
line 23428 processed
line 23429 processed
line 23430 processed
line 23431 processed
line 23432 processed
line 23433 processed
line 23434 processed
line 23435 processed
line 23436 processed
line 23437 processed
line 23438 processed
line 23439 processed
line 23440 processed
line 23441 processed
line 23442 processed
line 23443 processed
line 23444 processed
line 23445 processed
line 23446 processed
line 23447 processed
line 23448 processed
line 23449 processed
line 23450 processed
line 23451 processed
line 23452 processed
line 23453 processed
line 23454 processed
line 23455 processed
line 23456 processed
line 23457 processed
line 23458 processed
line 23459 processed
line 23460 processed
line 23461 processed
line 23462 processed
line 23463 pr

line 23829 processed
line 23830 processed
line 23831 processed
line 23832 processed
line 23833 processed
line 23834 processed
line 23835 processed
line 23836 processed
line 23837 processed
line 23838 processed
line 23839 processed
line 23840 processed
line 23841 processed
line 23842 processed
line 23843 processed
line 23844 processed
line 23845 processed
line 23846 processed
line 23847 processed
line 23848 processed
line 23849 processed
line 23850 processed
line 23851 processed
line 23852 processed
line 23853 processed
line 23854 processed
line 23855 processed
line 23856 processed
line 23857 processed
line 23858 processed
line 23859 processed
line 23860 processed
line 23861 processed
line 23862 processed
line 23863 processed
line 23864 processed
line 23865 processed
line 23866 processed
line 23867 processed
line 23868 processed
line 23869 processed
line 23870 processed
line 23871 processed
line 23872 processed
line 23873 processed
line 23874 processed
line 23875 processed
line 23876 pr

line 24229 processed
line 24230 processed
line 24231 processed
line 24232 processed
line 24233 processed
line 24234 processed
line 24235 processed
line 24236 processed
line 24237 processed
line 24238 processed
line 24239 processed
line 24240 processed
line 24241 processed
line 24242 processed
line 24243 processed
line 24244 processed
line 24245 processed
line 24246 processed
line 24247 processed
line 24248 processed
line 24249 processed
line 24250 processed
line 24251 processed
line 24252 processed
line 24253 processed
line 24254 processed
line 24255 processed
line 24256 processed
line 24257 processed
line 24258 processed
line 24259 processed
line 24260 processed
line 24261 processed
line 24262 processed
line 24263 processed
line 24264 processed
line 24265 processed
line 24266 processed
line 24267 processed
line 24268 processed
line 24269 processed
line 24270 processed
line 24271 processed
line 24272 processed
line 24273 processed
line 24274 processed
line 24275 processed
line 24276 pr

line 24630 processed
line 24631 processed
line 24632 processed
line 24633 processed
line 24634 processed
line 24635 processed
line 24636 processed
line 24637 processed
line 24638 processed
line 24639 processed
line 24640 processed
line 24641 processed
line 24642 processed
line 24643 processed
line 24644 processed
line 24645 processed
line 24646 processed
line 24647 processed
line 24648 processed
line 24649 processed
line 24650 processed
line 24651 processed
line 24652 processed
line 24653 processed
line 24654 processed
line 24655 processed
line 24656 processed
line 24657 processed
line 24658 processed
line 24659 processed
line 24660 processed
line 24661 processed
line 24662 processed
line 24663 processed
line 24664 processed
line 24665 processed
line 24666 processed
line 24667 processed
line 24668 processed
line 24669 processed
line 24670 processed
line 24671 processed
line 24672 processed
line 24673 processed
line 24674 processed
line 24675 processed
line 24676 processed
line 24677 pr

First, we separate the text column into its own numpy array. Then, we convert the numpy array into a list. This puts our data into a universal list format for when we clean the data.

Next, we initialize variables we need and start looping over the individual text descriptions.

We start by using regular expressions to get rid of punctuation and symbols, and we also make all of the text lower case. This makes sure repeated words are recognized in our text.

Next, we split our description string into words by splitting on the spaces between words. This allows us to modify individual words in our text.

Then, we remove stop words from our text and stem all our words. Stop words are words such as "and", "the", and "of" which do not have a lot of sentimental value to our machine learning. You can think of it as removing noise. However, you have to be careful. Removing stop words on a dataset with short phrases such as "On the blue river" and "Over the mountain" can cause a loss of important information.

When we stem all the words we are removing irrelevant part of words such as prefixes and suffixes which are similar. We use the Porter Stemmer, which is well known to be a good solution to stemming words. Stemming also tends to reduce the amount of words in our bag of words model later on, which can help lower the dimensionality of our input data. However, we need to be careful to not stem useful information like with removing stop words.

Finally, we join the modified words back together with spaces and keep track of our progress.

Next, we will vectorize our text descriptions and add our text features back into our input data.

In [7]:
text_array = np.array(text_list)
data_array = data.values

tfidf = TfidfVectorizer()
text_vect = tfidf.fit_transform(text_array)

print(data_array.shape)
print(text_vect.shape)

full_data = np.concatenate((text_vect.toarray(), data_array), axis=1)

print(full_data.shape)

(25000, 1649)
(25000, 16728)
(25000, 18377)


First, we convert the main dataframe into a numpy array and the modified text list we made into a numpy array. Then, we use sklearn's term frequency vectorizer to vectorize our text data. Rather than simply counting the words in each description, we weight words based on term frequency. This vectorizer keeps less useful repeated words from overshadowing more useful unique words.

Then, we append our vectorized text data onto our input data and observe the new shape of the data. The feature count may look very large compared to the number of examples. However, this is due to us having to cut down the number of examples from 150,000 to 25,000. The final feature count with 150,000 examples is only around 31,000, which suggests a good amount of repeated words.

You can find more information about tf-idf term weighting here: http://scikit-learn.org/stable/modules/feature_extraction.html

Next, we will split our data into train and test data concat the input data and labels together to fit EMADE's format.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(full_data, labels.values, test_size=0.33)

data_train = np.concatenate((X_train, y_train), axis=1)
data_test = np.concatenate((X_test, y_test), axis=1)

print(data_train.shape)
print(data_test.shape)

(16750, 18378)
(8250, 18378)


We split the data into training and test data with 67% as training and 33% as testing data. Then, we append the labels columns to the end of the input data columns to put the data into the format EMADE expects.

Next, we will test out the performance of our dataset on a logistic regression classifier to get a rough benchmark. Logistic Regression works well on text datasets and any datasets with a lot of discrete and/or Boolean (1,0) values.

In [9]:
classifier = LogisticRegression()

classifier.fit(X_train, np.ravel(y_train))

predicted = classifier.predict(X_test)

print("Classification report for classifier %s:\n%s\n"
      % (classifier, metrics.classification_report(y_test, predicted)))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(y_test, predicted))

Classification report for classifier LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False):
             precision    recall  f1-score   support

        0.0       0.89      0.96      0.93      6804
        1.0       0.72      0.46      0.56      1446

avg / total       0.86      0.87      0.86      8250


Confusion matrix:
[[6547  257]
 [ 785  661]]


Next, we will convert our train and test data back into pandas dataframes and export it in the proper format for EMADE.

In [10]:
train = pd.DataFrame(data_train)
test = pd.DataFrame(data_test)

divisor_train = math.ceil(len(train) / 5)
divisor_test = math.ceil(len(test) / 5)

count = 0
for g, df in train.groupby(np.arange(len(train)) // divisor_train):
    print(df.shape)

    np.savetxt("datasets/wine_data_set_train_%i.txt" % count, df.values, fmt='%.5f', delimiter=",")

    with open("datasets/wine_data_set_train_%i.txt" % count, "rb") as f_in, gzip.open("datasets/wine_data_set_train_%i.dat.gz" % count, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
        
    os.remove("datasets/wine_data_set_train_%i.txt" % count)

    count += 1
    
count = 0
for g, df in test.groupby(np.arange(len(test)) // divisor_test):
    print(df.shape)

    np.savetxt("datasets/wine_data_set_test_%i.txt" % count, df.values, fmt='%.5f', delimiter=",")

    with open("datasets/wine_data_set_test_%i.txt" % count, "rb") as f_in, gzip.open("datasets/wine_data_set_test_%i.dat.gz" % count, "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)
        
    os.remove("datasets/wine_data_set_test_%i.txt" % count)

    count += 1
    
small_train = train.drop(train.index[1675:])
small_test = test.drop(test.index[825:])

np.savetxt("datasets/wine_data_set_train_small.txt", small_train.values, fmt='%.5f', delimiter=",")

with open("datasets/wine_data_set_train_small.txt", "rb") as f_in, gzip.open("datasets/wine_data_set_train_small.dat.gz", "wb") as f_out:
    shutil.copyfileobj(f_in, f_out)
    
os.remove("datasets/wine_data_set_train_small.txt")
              
np.savetxt("datasets/wine_data_set_test_small.txt", small_train.values, fmt='%.5f', delimiter=",")

with open("datasets/wine_data_set_test_small.txt", "rb") as f_in, gzip.open("datasets/wine_data_set_test_small.dat.gz", "wb") as f_out:
    shutil.copyfileobj(f_in, f_out)
    
os.remove("datasets/wine_data_set_test_small.txt")

(3350, 18378)
(3350, 18378)
(3350, 18378)
(3350, 18378)
(3350, 18378)
(1650, 18378)
(1650, 18378)
(1650, 18378)
(1650, 18378)
(1650, 18378)


We split the dataset into 5 chunks each with 20% of the data. Then, we set aside one chunk of 10% to be our small dataset in EMADE.

The final step for preprocessing is to compress all the .dat files into .dat.gz files and place them in a folder under the datasets directory of EMADE.

There are other datasets already implemented into EMADE in this format.

# XML Formatting

Below is an example xml template for the dataset we preprocessed. This template is used to setup parameters for EMADE. You can change the objectives, crossover probability, and mutation probability from here.

Make sure to make an xml file and store it in the templates directory when you implement a new dataset into EMADE.

## Note: Do NOT run the code cell below

In [11]:
<?xml version="1.0"?>

<input>

    <datasets>
        <dataset>
            <name>SmallDataSet</name>
            <type>featuredata</type>
            <MonteCarlo>
                <trial>
                    <trainFilename>datasets/wine/wine_data_set_train_small.dat.gz</trainFilename>
                    <testFilename>datasets/wine/wine_data_set_test_small.dat.gz</testFilename>
                </trial>
            </MonteCarlo>
        </dataset>
        <dataset>
            <name>FullDataSet</name>
            <type>featuredata</type>
            <MonteCarlo>
                <trial>
                    <trainFilename>datasets/wine/wine_data_set_train_0.dat.gz</trainFilename>
                    <testFilename>datasets/wine/wine_data_set_test_0.dat.gz</testFilename>
                </trial>
                <trial>
                    <trainFilename>datasets/wine/wine_data_set_train_1.dat.gz</trainFilename>
                    <testFilename>datasets/wine/wine_data_set_test_1.dat.gz</testFilename>
                </trial>
                <trial>
                    <trainFilename>datasets/wine/wine_data_set_train_2.dat.gz</trainFilename>
                    <testFilename>datasets/wine/wine_data_set_test_2.dat.gz</testFilename>
                </trial>
                <trial>
                    <trainFilename>datasets/wine/wine_data_set_train_3.dat.gz</trainFilename>
                    <testFilename>datasets/wine/wine_data_set_test_3.dat.gz</testFilename>
                </trial>
                <trial>
                    <trainFilename>datasets/wine/wine_data_set_train_4.dat.gz</trainFilename>
                    <testFilename>datasets/wine/wine_data_set_test_4.dat.gz</testFilename>
                </trial>
            </MonteCarlo>
        </dataset>
    </datasets>

    <objectives>
        <objective>
            <name>False Positives</name>
            <weight>-1.0</weight>
            <achievable>4971.8</achievable>
            <goal>0</goal>
            <evaluationFunction>false_positive</evaluationFunction>
            <lower>0</lower>
            <upper>1</upper>
        </objective>
        <objective>
            <name>False Negatives</name>
            <weight>-1.0</weight>
            <achievable>1541.2</achievable>
            <goal>0</goal>
            <evaluationFunction>false_negative</evaluationFunction>
            <lower>0</lower>
            <upper>1</upper>
        </objective>
        <objective>
            <name>F1-Score</name>
            <weight>-1.0</weight>
            <achievable>0.2</achievable>
            <goal>0</goal>
            <evaluationFunction>f1_score_min</evaluationFunction>
            <lower>0</lower>
            <upper>1</upper>
        </objective>
        <objective>
            <name>Num Elements</name>
            <weight>-1.0</weight>
            <achievable>100.0</achievable>
            <goal>0</goal>
            <evaluationFunction>num_elements_eval_function</evaluationFunction>
            <lower>0</lower>
            <upper>1</upper>
        </objective>

    </objectives>

    <evaluation>
        <module>evalFunctions</module>
        <memoryLimit>30</memoryLimit> <!-- In Percent -->
    </evaluation>

    <scoopParameters>
        <host>
            <name>localhost</name>
            <workers>24</workers>
        </host>
        <!--<host>
            <name>localhost</name>
            <workers>3</workers>
        </host>
        <host>
            <name>localhost</name>
            <workers>3</workers>
        </host>
        <host>
            <name>localhost</name>
            <workers>3</workers>
        </host>
        <host>
            <name>localhost</name>
            <workers>3</workers>
        </host>
        <host>
            <name>localhost</name>
            <workers>3</workers>
        </host>-->
    </scoopParameters>

    <evolutionParameters>
        <initialPopulationSize>512</initialPopulationSize>
        <elitePoolSize>512</elitePoolSize>
        <launchSize>300</launchSize>
        <minQueueSize>200</minQueueSize>

        <matings>
            <mating>
                <name>crossover</name>
                <probability>0.50</probability>
            </mating>
            <mating>
                <name>crossoverEphemeral</name>
                <probability>0.50</probability>
            </mating>
            <mating>
                <name>headlessChicken</name>
                <probability>0.10</probability>
            </mating>
            <mating>
                <name>headlessChickenEphemeral</name>
                <probability>0.10</probability>
            </mating>
        </matings>

        <mutations>
            <mutation>
                <name>insert</name>
                <probability>0.05</probability>
            </mutation>
            <mutation>
                <name>insert modify</name>
                <probability>0.10</probability>
            </mutation>
            <mutation>
                <name>ephemeral</name>
                <probability>0.25</probability>
            </mutation>
            <mutation>
                <name>node replace</name>
                <probability>0.05</probability>
            </mutation>
            <mutation>
                <name>uniform</name>
                <probability>0.05</probability>
            </mutation>
            <mutation>
                <name>shrink</name>
                <probability>0.05</probability>
            </mutation>
        </mutations>

        <selections>
            <selection>
                <name>NSGA2</name>
            </selection>
        </selections>

    </evolutionParameters>

    <seedFile>

    </seedFile>

    <genePoolFitness>
        <prefix>genePoolFitnessWine</prefix>
    </genePoolFitness>
    <paretoFitness>
        <prefix>paretoFitnessWine</prefix>
    </paretoFitness>
    <parentsOutput>
        <prefix>parentsWine</prefix>
    </parentsOutput>



    <paretoOutput>
        <prefix>paretoFrontWine</prefix>
    </paretoOutput>
</input>

SyntaxError: invalid syntax (<ipython-input-11-ef8ab20c933c>, line 1)

In [ ]:
import nltk

In [ ]:
nltk.download()